# DISCOGS SCRAPPER

_Scraps discogs wishlist from user to find the seller who has the most items_

In [1]:
import numpy as np
import urllib.request
import html   
from operator import itemgetter
from itertools import groupby
import urllib

1. get url for wantlist from username

In [49]:
def wantlist(username):
    #username : username
    #RETURN : url as string
    
    
    a='https://www.discogs.com/sell/mywants?sort=listed%2Cdesc&limit=250&ev=wfs&user='+str(username)

    return a

2. explore html page

In [39]:
def get_array_mult(url):
    
    count=0
    
    url0 = url + '&page=1'
    
    #-------- get html code -----------#
    
    fp = urllib.request.Request(url0,headers={'User-Agent': 'Chrome'})
    ab = urllib.request.urlopen(fp)
    mybytes = ab.read()

    mystr = mybytes.decode("utf8")
    
    
    # ----------- num page ----------- #
    
    index_num=[i for i in range(len(mystr)) if mystr.startswith('250 of', i)]
    
    z=mystr[index_num[0]:index_num[0]+150]
        
    start = '250 of '
    end = '    </strong>'

    num_pages=z[z.find(start)+len(start):z.rfind(end)]
    
    print(num_pages)
    
    pages_num=int(num_pages.replace(',',''))//250+1
    
    # ------------------ listing -------------- #
    
    list_=[]
    
    print('number of pages ----- >',pages_num)
    
    for k in range(pages_num): 
        
        url_1 = url+'&page='+str(k+1)
        
        print(url_1)
                
        #-------- get html code -----------#
    
        fp_1 = urllib.request.Request(url_1,headers={'User-Agent': 'Chrome'})
        ab_1 = urllib.request.urlopen(fp)
        mybytes = ab_1.read()
        

        mystr = mybytes.decode("utf8")
        
        
        index_vendor  = [i for i in range(len(mystr)) if mystr.startswith('<strong><a href="/seller/', i)]
        index_name= [i for i in range(len(mystr)) if mystr.startswith('item_description_title', i)]


        print(len(index_name),len(index_vendor))

        # ----------- list ----------- #

        for i in range(len(index_name)):

            s=mystr[index_name[i]:index_name[i]+100]

            start = '>'
            end = '</a>'

            name=s[s.find(start)+len(start):s.rfind(end)]

            x=mystr[index_vendor[i]+20:index_vendor[i]+45]

            start = 'ler/'
            end = '/prof'

            seller=x[x.find(start)+len(start):x.rfind(end)]

            list_.append([seller,name])

            count=count+1
        # -------------- kick the double ------------ #

    list_ = [x for n, x in enumerate(list_) if x not in list_[:n]]
        
    print('Element added : ',count)
    return(list_)    
        

3. Testing with nerxunwaer

In [50]:
list_test=get_array_mult(wantlist('nerxunwaer'))

430
number of pages ----- > 2
https://www.discogs.com/sell/mywants?sort=listed%2Cdesc&limit=250&ev=wfs&user=nerxunwaer&page=1
250 250
https://www.discogs.com/sell/mywants?sort=listed%2Cdesc&limit=250&ev=wfs&user=nerxunwaer&page=2
250 250
Element added :  500


4. Group element then search the index, count every instance, then apply

In [41]:
def result(list_):
    #return the list with the result
    list_res=[]
    for grp, elmts in groupby(list_, itemgetter(0)):
        list_res.append((grp, len(list(elmts))))
    list_res=sorted(list_res,key=itemgetter(1),reverse=True)
    return list_res

In [42]:
def search_list(string, list_,lenght):
    #return the index of the searched string
    
    index = -1
    
    for k in range(lenght):
        
        if str(list_[k][0])==string : 
            
            index=k
            
            break

    return index      

In [44]:
def super_count_vynil(list_finale):
    # ---- count & save each instance, return list of triple #
    
    list_hello=[]

    for k in range(len(list_finale)): 

        index = search_list(str(list_finale[k][0]),list_hello,len(list_hello))

        if index==-1: 

            list_hello.append([list_finale[k][0],1, [list_finale[k][1]]])

        if index>-1:

            list_hello[index][1]=list_hello[index][1]+1

            list_hello[index][2].append(list_finale[k][1])
        
    return list_hello

In [52]:
a=super_count_vynil(list_test)

5. Sort the result and results

In [ ]:
def sort_list(list_): 
    
    #return the sorted list for index [1]
    
    big_list=[]
    
    for k in range(len(list_)):
        
        big_list.append((list_[k][1],k))
    
    big_list=sorted(big_list,reverse=True)
    
    new_list=[]
    
    for k in range(len(list_)):
        
        ind=big_list[k][1]
        
        new_list.append(list_[ind])
    
    return new_list

In [47]:
sort_list(a)[0:10]

[['recordsale-de',
  3,
  ['Head High - Megatrap (2x12")',
   'Various - Remix Culture 155 (2x12")',
   'Crustation - Flame (12")']],
 ['SandkastenVinylstor',
  3,
  ['Head High - Megatrap (2x12")',
   'Leon Vynehall - Music For The Uninvited  (CD, Album)',
   'Crustation - Flame (12", Single, Ltd, RE)']],
 ['Drogadiscos',
  2,
  ['Andronicus - Make You Whole (12")',
   'Andronicus - Make You Whole (12", Promo)']],
 ['tammy731605',
  2,
  ['Andronicus - Make You Whole (12", Gen)',
   'Andronicus - Make You Whole (12")']],
 ['DELAROSA_PLASTICS/p',
  2,
  ['Soul II Soul - Pleasure Dome (Booker T Mixes) (12", ',
   'Andronicus - Make You Whole (Remixes) (2x12")']],
 ['decks.de',
  2,
  ['Leon Vynehall - Music For The Uninvited  (CD, Album)',
   'Leon Vynehall - Nothing Is Still (CD, Album)']],
 ['r.brown600',
  2,
  ['Andronicus - Make You Whole (12")', 'Crustation - Flame (2x12", Promo)']],
 ['alain.v2',
  2,
  ['Leon Vynehall - Music For The Uninvited  (2x12", Alb',
   'Head High - Mega